# CAFQA

In [4]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize
from clapton.ansatzes import circular_ansatz,circular_ansatz_mirrored
import numpy as np

In [5]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
coeffs = np.random.random(len(paulis))

In [6]:
# define parametrized Clifford circuit that is being optimized over
# here we use the circular_ansatz template
# we fix 2q gates as they will not be optimized over
vqe_pcirc = circular_ansatz_mirrored(N=len(paulis[0]), reps=1, fix_2q=True)

In [7]:
# the circuit consists of parametrized gates
for gate in vqe_pcirc.gates:
    print(gate.label, gate.is_fixed())

RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False
2Q True
2Q True
2Q True
RY False
RY False
RY False
RZ False
RZ False
RZ False


In [8]:
# non-fixed gates will be optimized over
# RY and RZ gates can assume 4 values k = 0,1,2,3 which describe multiples of pi/2

In [9]:
# the initial parameters are all 0
vqe_pcirc.read()

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
# we can look at the corresponding stim circuit
vqe_pcirc.stim_circuit().diagram()

q0: -I-I-X-@-I-I-----@-X-I-I-
         | |         | |
q1: -I-I-|-X-@-I-I-@-X-|-I-I-
         |   |     |   |
q2: -I-I-@---X-I-I-X---@-I-I-

In [73]:
# we can assign a different set of parameters
vqe_pcirc.assign([0,1,2,3,0,1,2,3,0,1,2,3])

In [74]:
vqe_pcirc.stim_circuit().diagram()

q0: -I------S_DAG-X-@-Y-S----------------
                  | |
q1: -SQRT_Y-I-----|-X-@-SQRT_Y_DAG-Z-----
                  |   |
q2: -Y------S-----@---X-I----------S_DAG-

In [75]:
# we can perform CAFQA by using the main optimization function "claptonize"
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs


started GA at id None with 1 procs

started GA at id 3 with 1 procs
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2], dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2], dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2], dtype=object)]
[0, array([0., 0., 0., 0.]), array([3, 3, 0, 2, 3, 3, 2, 3, 2, 1, 1, 2], dtype=object)]
[1, array([0., 0., 0., 0.]), array([0, 2, 1, 1, 2, 3, 0, 1, 2, 0, 0, 3], dtype=object)]
[1, array([0., 0., 0., 0.]), array([0, 2, 1, 1, 2, 3, 0, 1, 2, 0, 0, 3], dtype=object)]
[1, array([0., 0., 0., 0.]), array([0, 2, 1, 1, 2, 3, 0, 1, 2, 0, 0, 3], dtype=object)]
[1, array([0., 0., 0., 0.]), array([0, 2, 1, 1, 2, 3, 0, 1, 2, 0, 0, 3], dtype=object)]
[2, array([0., 0., 0., 0.]), array([0, 2, 1, 1, 2, 3, 0, 1, 2, 0, 0, 3], dtype=object)]
[2, array([0., 0., 0., 0.]), array([0, 2, 1, 1, 2, 3, 

In [76]:
# the best parameters are
ks_best

[1, 2, 0, 0, 0, 0, 3, 2, 3, 0, 3, 0]

In [77]:
# with energy
energy_best

0.0

In [78]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.stim_circuit().diagram()

q0: -SQRT_Y-I-X-@-SQRT_Y_DAG-I----------------
              | |
q1: -Y------I-|-X-@----------Y----------S_DAG-
              |   |
q2: -I------I-@---X----------SQRT_Y_DAG-I-----

# noisy CAFQA (nCAFQA)

In [79]:
from clapton.depolarization import GateGeneralDepolarizationModel #TODO: check this out 

In [80]:
# let's add a noise model where we specify global 1q and 2q gate errors
nm = GateGeneralDepolarizationModel(p1=0.005, p2=0.1) #NOTE: This is the Noise Model, not representative of device noise

In [81]:
vqe_pcirc = circular_ansatz(N=len(paulis[0]), reps=1, fix_2q=True)
vqe_pcirc.add_depolarization_model(nm)

In [82]:
# after every gate a depol channel is added
vqe_pcirc.stim_circuit().diagram()

q0: -I-DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------X-DEPOLARIZE2(0.1)-@-DEPOLARIZE2(0.1)---I----------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)---------------------------------------
                                                                                                                         | |                | |
q1: ---I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------|-|----------------X-DEPOLARIZE2(0.1)-@-DEPOLARIZE2(0.1)-I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)--------------------
                                                                                                                         | |                                   | |
q2: ----------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-@-DEPOLARIZE2(0.1)--------------------X-DEPOLARIZE2(0.1)--------------------I------------------DEPOLARIZE1(0.005)--------------------I------------------DEPOLARIZE1(0.005)-

In [83]:
# we can perform nCAFQA by using the main optimization function "claptonize"
# now with the noisy circuit
ks_best, energy_noisy, energy_noiseless = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)

STARTING ROUND 0


started GA at id 1 with 1 procs

started GA at id 2 with 1 procs

started GA at id None with 1 procs

started GA at id 3 with 1 procs

[0, array([-0.02620387, -0.02620387,  0.        ,  0.        ]), array([0, 2, 0, 0, 0, 2, 2, 1, 1, 2, 1, 0], dtype=object)]
[0, array([-0.02538953, -0.02538953,  0.        ,  0.        ]), array([2, 1, 2, 0, 1, 2, 0, 0, 0, 1, 2, 2], dtype=object)]
[0, array([-0.03177963, -0.03177963,  0.        ,  0.        ]), array([2, 1, 2, 0, 1, 2, 0, 0, 0, 1, 2, 2], dtype=object)]
[0, array([-0.0280677, -0.0280677,  0.       ,  0.       ]), array([0, 1, 1, 0, 3, 2, 2, 1, 1, 0, 3, 3], dtype=object)]
[1, array([-0.02972533, -0.02972533,  0.        ,  0.        ]), array([2, 1, 2, 0, 0, 1, 1, 0, 3, 1, 0, 0], dtype=object)]
[1, array([-0.02838842, -0.02838842,  0.        ,  0.        ]), array([2, 3, 0, 0, 2, 0, 2, 1, 2, 3, 3, 2], dtype=object)]
[1, array([-0.03756204, -0.03756204,  0.        ,  0.        ]), array([0, 1, 1, 0, 3, 0, 0, 2, 2, 2, 0, 3

In [84]:
# the best parameters are
ks_best

[2, 1, 1, 2, 3, 3, 0, 0, 0, 0, 0, 3]

In [85]:
# with noisy/noiseless energy
energy_noisy, energy_noiseless

(0.002429814460377592, 0.0)

In [86]:
# differrence
np.abs(energy_noisy-energy_noiseless)

0.002429814460377592

In [87]:
# the corresponding circuit is
vqe_pcirc.assign(ks_best)
vqe_pcirc.snapshot_noiseless().circ_snapshot_noiseless.diagram()

q0: -Y------Z-----X-@-I-I-------
                  | |
q1: -SQRT_Y-S_DAG-|-X-@-I-I-----
                  |   |
q2: -SQRT_Y-S_DAG-@---X-I-S_DAG-